<a href="https://colab.research.google.com/github/jepilogo97/nlp/blob/main/chatbot-rag/chatbot_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ChatBot - RAG

##### Jean Pierre Londoño González
##### Mini-Proyecto de chatbot utilizando RAG
##### 28SEP2025

Partiendo de los conceptos establecidos en el [1-ollama-rag.ipynb](./1-ollama-rag.ipynb) ahora exploraremos una implementación con herramientas más maduras que facilitan el manejo de RAGs y LLMs. Además de eso, continuaremos el uso de [Ollama](https://ollama.com) y añadiremos el uso de [LangChain](https://www.langchain.com) un framework orientado al desarrollo de LLMs y agentes de IA. El objetivo final es el mismo, crear un RAG a partir de los documentos de wikihow. Observaremos que algunos de los pasos del notebook anterior se hacen de forma explicita.

**Nota:** En este notebook no se realizará el entrenamiento de ningún modelo, se utilizarán modelos pre-entrenados y se combinaran de una forma interesante para hacer uso de cada componente a su modo y crear un agente de conversación

#### Referencias
- Dataset: https://huggingface.co/datasets/RamAnanth1/lex-fridman-podcasts
- https://huggingface.co/EleutherAI/gpt-neo-125m

### 1. Importación de librerias y carga de modelos

Inicio importando las librerías necesarias para el procesamiento de lenguaje natural, la manipulación de datos y la construcción del modelo. Esto incluye NumPy y pandas para el manejo y análisis de datos; Hugging Face Datasets y Transformers para la carga de corpus y la tokenización; y PyTorch junto con PyTorch Lightning para definir, entrenar y evaluar el modelo de manera estructurada.

In [1]:
import pkg_resources
import warnings

warnings.filterwarnings('ignore')

installed_packages = [package.key for package in pkg_resources.working_set]
IN_COLAB = 'google-colab' in installed_packages

/tmp/ipython-input-2396000874.py:1: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


In [2]:
!wget -O requirements.txt https://raw.githubusercontent.com/jepilogo97/nlp/main/chatbot-rag/requirements.txt
!pip install -r requirements.txt

--2025-09-28 20:37:14--  https://raw.githubusercontent.com/jepilogo97/nlp/main/chatbot-rag/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 449 [text/plain]
Saving to: ‘requirements.txt’

requirements.txt    100%[===================>]     449  --.-KB/s    in 0s      

2025-09-28 20:37:14 (13.8 MB/s) - ‘requirements.txt’ saved [449/449]

  Using cached matplotlib-3.9.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached huggingface_hub-0.24.6-py3-none-any.whl.metadata (13 kB)
  Using cached sentence_transformers-3.0.1-py3-none-any.whl.metadata (10 kB)
  Using cached ollama-0.1.7-py3-none-any.whl.metadata (3.8 kB)
  Using cached langchain-0.2.16-py3-none-any.whl.metadata (7.1 kB)
ERROR: Ignored the followi

In [3]:
# =======================
# Procesamiento de datos y utilidades
# =======================
import numpy as np  # Cálculo numérico eficiente y manejo de arreglos multidimensionales
import pandas as pd  # Manipulación y análisis de datos en estructuras tipo DataFrame

# Configuración de pandas para mostrar todos los datos en consola sin recorte
pd.set_option("display.max_rows", None)     # Muestra todas las filas del DataFrame
pd.set_option("display.max_columns", None)  # Muestra todas las columnas
pd.set_option("display.width", None)        # Evita el corte de líneas largas al imprimir

# =======================
# Manejo de datasets de Hugging Face
# =======================
from datasets import Dataset, load_dataset, concatenate_datasets  # Carga, creación y combinación de datasets
from collections import Counter  # Conteo de frecuencias de elementos (tokens, palabras, etc.)

import matplotlib.pyplot as plt   # Visualización de datos en gráficos estáticos

# =======================
# Tipado y utilidades para funciones
# =======================
from typing import Callable, Dict, List, Optional, Any, Generator  # Anotaciones de tipo para mayor claridad y validación
from abc import abstractmethod                                     # Definición de métodos abstractos en clases base

# =======================
# Progreso en bucles y NLP con Transformers
# =======================
from tqdm.auto import tqdm  # Barra de progreso adaptable a terminal o Jupyter

# =======================
# Sistema y utilidades
# =======================
import os                  # Operaciones del sistema de archivos y variables de entorno
from huggingface_hub import login  # Autenticación y subida/descarga de modelos/datasets a Hugging Face Hub
from pathlib import Path   # Manejo de rutas de archivos y directorios de forma multiplataforma
import time               # Medición de tiempos y pausas en procesos
import random             # Generación de números aleatorios

# =======================
# Modelos de embeddings y búsqueda semántica
# =======================
from sentence_transformers import SentenceTransformer, util as st_util  # Generación y comparación de embeddings de texto
import ollama  # Cliente para interactuar con modelos locales/servidores de Ollama

# =======================
# Integración con LangChain
# =======================
from langchain_ollama import ChatOllama  # Conector LangChain para modelos de Ollama
from langchain_community.vectorstores import FAISS  # Almacenamiento de vectores (búsqueda semántica) usando FAISS
from langchain_huggingface import HuggingFaceEmbeddings  # Embeddings usando modelos de Hugging Face
from langchain.prompts import PromptTemplate  # Plantillas para prompts
from langchain.chains.combine_documents import create_stuff_documents_chain  # Encadenamiento de documentos en consultas
from langchain_core.prompts import ChatPromptTemplate  # Plantillas de chat estructuradas
from langchain.chains import (
    create_history_aware_retriever,  # Recuperador con contexto de conversación previa
    create_retrieval_chain,          # Cadena de recuperación de información
    RetrievalQA                      # Pipeline de preguntas y respuestas sobre documentos
)
from langchain.schema import HumanMessage, AIMessage  # Representación de mensajes en el flujo conversacional

# =======================
# Interfaz web
# =======================
import gradio as gr  # Creación de interfaces web ligeras para demos y aplicaciones de ML


ModuleNotFoundError: No module named 'ollama'

### 2. Exploración de dataset

Se utilizará el dataset RagQuAS, el cuál contiene ejemplos en una gran cantidad de dominios: Hobbies, Lingüística, Mascotas, Salud, astronomía, atención al cliente, coches, cotidiano, documentación, energía, esquí, estafas, gastronomía, hobbies, idiomas, juegos, lenguaje, manicura, música, patinaje, primeros auxilios, receta, reciclaje, reclamaciones, seguros, tenis, transporte, turismo, veterinaria, viajes, yoga.

- https://huggingface.co/datasets/IIC/RagQuAS

In [ ]:
login()

In [ ]:
dataset = load_dataset('IIC/RagQuAS', split='test')
dataset

In [ ]:
pd.set_option('display.max_colwidth', 100)
dataset.set_format(type='pandas')
df = dataset.to_pandas()
df.head(15)

In [ ]:
word_counts = (
    df[["text_1", "text_2", "text_3", "text_4", "text_5"]]
    .apply(lambda row: sum(len(str(text).split()) for text in row), axis=1)
)

# Graficar la distribución
fig, ax = plt.subplots(figsize=(10, 6))
word_counts.hist(ax=ax, bins=30, edgecolor='black')
ax.set_title("Distribución del número total de palabras por documento")
ax.set_xlabel("Número de palabras")
ax.set_ylabel("Frecuencia")
plt.show()

En este notebook usarémos LangChain y FAISS para crear el document store. Por ahora vamos a saltarnos el paso de chunking ya que esta combinación de librerías realiza lo suficientemente bien la tarea con los documentos enteros.

### 2. Carga de modelo

In [ ]:
!if ! type ollama > /dev/null; then curl -fsSL https://ollama.com/install.sh | sh; else echo "Ollama ya está instalado."; fi

In [ ]:
%load_ext colabxterm
%xterm

Para este caso, vamos a trabajar con un modelo más moderno y pequeño: llama3.2:3b

In [ ]:
!ollama pull llama3.2:3b

In [ ]:
llm = ChatOllama(model="llama3.2:3b", validate_model_on_init=True)
llm.invoke("El primer hombre en la luna fue...").content

### 3. Creando el document store

Ahora vamos a crear el document store. Para ello, utilizaremos la librería FAISS (a través de LangChain) que está especializada para la indexación y búsqueda de vectores.

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")
texts = [t[0] for t in documents['text'].to_list()]

index_path = './faiss_index'
if os.path.exists(index_path):
    vectorstore = FAISS.load_local(index_path, embeddings, allow_dangerous_deserialization=True)
else:

    vectorstore = FAISS.from_texts(texts, embeddings)
    vectorstore.save_local(index_path)

retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

### 4. Poniendo a prueba un QA simple


Ahora configuamos un prompt y un RetrievalQA para nuestro caso. Aquí observamos la simpleza para crear la interfaz. Esta es una de las formas de crearla. Esta forma, según la librería está deprecada y podría ser removida en el futuro.

In [ ]:
prompt = PromptTemplate.from_template(
    """Utiliza los siguientes fragmentos de contexto para responder la pregunta al final.
    Si no sabes la respuesta, di que no lo sabes.
    No menciones que te he proporcionado fragmentos, simula que ya tenías esta información en tu conocimiento y responde como en una conversación natural.
    Incluye las citas a las fuentes de información

    {context}

    Pregunta: {question}
    Respuesta Útil:"""
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)

result = qa_chain({"query": "Qué es el feng shui?"})

print(result["result"])
for i, doc in enumerate(result["source_documents"], 1):
    idx = vectorstore.index.reconstruct_n(0, vectorstore.index.ntotal)[0]
    try:
        original_idx = texts.index(doc.page_content)
    except ValueError:
        original_idx = None

    if original_idx is not None:
        meta = documents[original_idx]
        print(f"- [{i}] {meta['title'][0]} ({meta['url'][0]})")
    else:
        print(f"- [{i}] Unknown source: {doc.page_content[:50]}...")

Ahora lo intentamos de nuevo con la forma recomendada. Observamos que no es muy diferente, salvo el template y el nombre de los inputs. Otra ventaja de esta forma es que en la respuesta obtenemos automáticamente el contexto en forma de los documentos que fueron recuperados para construir la respuesta. Ya no es necesario pasar explicitamente una propiedad para recuperarlos.

In [ ]:
prompt = PromptTemplate.from_template(
    """Utiliza los siguientes fragmentos de contexto para responder la pregunta al final.
    Si no sabes la respuesta, di que no lo sabes.
    No menciones que te he proporcionado fragmentos, simula que ya tenías esta información en tu conocimiento y responde como en una conversación natural.
    Incluye las citas a las fuentes de información

    {context}

    Pregunta: {input}
    Respuesta Útil:"""
)

def format_answer(reply):
  answer = reply["answer"] + "\n"
  for i, doc in enumerate(reply["context"], 1):
    idx = vectorstore.index.reconstruct_n(0, vectorstore.index.ntotal)[0]
    try:
        original_idx = texts.index(doc.page_content)
    except ValueError:
        original_idx = None

    if original_idx is not None:
        meta = documents[original_idx]
        answer += f"- [{i}] {meta['title'][0]} ({meta['url'][0]})\n"
    else:
        answer += f"- [{i}] Unknown source: {doc.page_content[:50]}...\n"
  return answer

combine_docs_chain = create_stuff_documents_chain(llm, prompt)
qa_chain = create_retrieval_chain(retriever, combine_docs_chain)

result = qa_chain.invoke({"input": "Qué es el feng shui?"})

print(format_answer(result))

Y una vez más, observamos que el artículo del feng shui aparece en las referencias obtenidas.

Ahora, si hacemos una nueva pregunta observamos que las referencias cambian. Es buena señal de que nuestro RAG está funcionando como se debe.

In [ ]:
result = qa_chain.invoke({"input": "Cómo usar un computador?"})
print(format_answer(result))

Vemos que por lo menos dentro de los resultados hemos recuperado el articulo sobre feng shui, lo cual nos indica que el retriever si podría entregarnos resultados relevantes.

### 5. Creando una cadena conversacional

El ejemplo anterior sirve más que todo para responder únicas preguntas, no guarda el historial de conversación por lo que la cadena no puede usarla como parte del contexto.

Esto lo podemos arreglar mediante un history retrieval:

In [ ]:
condense_question_system_template = (
    "Utiliza el historial de conversación y fragmentos de contexto para reformular la pregunta al final sin tener que incluir todo el historial."
    "No menciones que te he proporcionado fragmentos, simula que ya tenías esta información en tu conocimiento y responde como en una conversación natural."
)

condense_question_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", condense_question_system_template),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    llm, retriever, condense_question_prompt
)

system_prompt = (
    "Eres un asistente para tareas de tipo preguntas y respuestas."
    "Utiliza las siguientes piezas del contexto recuperado para responder la pregunta."
    "Si no sabes la respuesta, di que no lo sabes."
    "\n\n"
    "{context}"
)

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("placeholder", "{chat_history}"),
        ("human", "{input}")
    ]
)

qa_chain = create_stuff_documents_chain(llm, qa_prompt)
convo_qa_chain = create_retrieval_chain(history_aware_retriever, qa_chain)

chat_history = []
response = convo_qa_chain.invoke(
    {
        "input": "qué es el feng shui?",
        "chat_history": chat_history,
    }
)

# Aquí debemos ir construyendo el historial para pasarlo a subsecuentes invocaciones
chat_history.append(HumanMessage(content=response["input"]))
chat_history.append(AIMessage(content=response["answer"]))
print(format_answer(response))



In [ ]:
response

### 6. Lanzando la interfáz de usuario del ChatBot

Finalmente, utilizarémos gradio nuevamente para construir la interfáz conversacional.

Nótese el manejo que le damos al historial de la conversación. Tenémos dos:

1. `lc_history`: Exclusivo al historial de LangChain, contiene estructuras propias de LangChain
2. `chat_history`: Es el historial de la interfaz de gradio, es mas simple y no debe mezclarse con el otro.

In [ ]:
convo_qa_chain = create_retrieval_chain(history_aware_retriever, qa_chain)
lc_history = []

with gr.Blocks() as gr_blocks:
    chatbot = gr.Chatbot()
    msg = gr.Textbox(
        label="Sobre qué quieres conversar?",
        placeholder="Ház tu pregunta aquí y presiona enter."
    )
    clear = gr.Button("Limpiar")

    def respond(question, chat_history):
        reply = convo_qa_chain.invoke({"input": question, "chat_history": lc_history})

        lc_history.append(HumanMessage(content=question))
        lc_history.append(AIMessage(content=reply["answer"]))

        answer = format_answer(reply)
        chat_history.append((question, answer))
        return "", chat_history

    def reset_chat():
        lc_history.clear()
        return ""

    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    clear.click(reset_chat, None, chatbot, queue=False)

gr_blocks.launch(inline=False)

In [ ]:
gr_blocks.close()

### 7. Conclusiones

#### Eficacia del flujo de análisis

- Al comparar arquitecturas de modelos de lenguaje como BERT y GPT, se identifican similitudes en su capacidad de generar representaciones ricas del lenguaje, algunas diferencias clave en su estructura y en su proceso de entrenamiento.

- Ambos modelos demuestran que un pre-entrenamiento sólido y la construcción de embeddings de alta calidad son factores críticos para alcanzar buenos resultados en tareas posteriores, evitando costos de entrenamiento desde cero.

#### Rendimiento del modelo

- Tanto BERT como GPT pueden adaptarse a una amplia variedad de tareas posteriores (clasificación, generación de texto, análisis semántico, etc.), lo que valida la versatilidad de los enfoques de transfer learning y fine tuning.

- La elección del modelo y la estrategia de entrenamiento depende de la tarea: BERT sobresale en comprensión y análisis de contexto bidireccional, mientras que GPT destaca en generación de texto coherente y fluido.

#### Limitaciones observadas

- Los modelos generativos enfrentan un dilema de exploración–explotación: una decodificación enfocada en la explotación (p. ej. greedy search) brinda mayor precisión pero tiende a producir textos monótonos; en cambio, la exploración (p. ej. sampling con temperatura alta) promueve creatividad y diversidad, pero con riesgo de incoherencia o “alucinaciones”.

- La calidad del modelo depende en gran medida de los datos de entrenamiento. Conjuntos de datos sesgados, poco representativos o de baja calidad pueden degradar el desempeño e introducir sesgos o errores difíciles de corregir.

#### Áreas de mejora

- Profundizar en la selección y curaduría de datasets, asegurando diversidad, equilibrio y relevancia para el dominio de aplicación.

- Experimentar con estrategias de decodificación y con la afinación de hiperparámetros para encontrar el punto óptimo entre creatividad, coherencia y precisión.

- Explorar técnicas de optimización y compresión que permitan desplegar modelos grandes en entornos de recursos limitados.

#### Valor práctico

- La adopción de modelos pre-entrenados como GPT ofrece una base sólida y flexible para proyectos de NLP, equilibrando costo, tiempo y calidad.

- El entendimiento de los trade-offs entre exploración y explotación, así como la adecuada selección de datos y métodos de decodificación, es esencial para alinear el modelo con los objetivos específicos del negocio y minimizar riesgos de sesgos o resultados no deseados.

### 8. Apendice

In [ ]:
import pkg_resources

libs = [
    "numpy",
    "pandas",
    "datasets",
    "torch",
    "pytorch-lightning",
    "torchmetrics",
    "tqdm",
    "transformers",
    "scikit-learn"
]

for lib in libs:
    try:
        version = pkg_resources.get_distribution(lib).version
        print(f"{lib}=={version}")
    except Exception:
        print(f"{lib}")

In [ ]:
 ## Solo correr en local

# import nbformat

## Cargar notebook
# with open("nlp_with_gpt.ipynb", "r", encoding="utf-8") as f:
    # nb = nbformat.read(f, as_version=4)

## Eliminar widgets corruptos si existen
# if "widgets" in nb["metadata"]:
    # del nb["metadata"]["widgets"]

## Guardar reparado
# with open("nlp_with_gpt.ipynb", "w", encoding="utf-8") as f:
    # nbformat.write(nb, f)